# 🧠 Week 7-8 · Notebook 01 · LangChain Essentials: Building the Copilot's Foundation

**Module:** LangChain, Agents, & Advanced RAG  
**Project:** Manufacturing Copilot - Core Pipeline Construction

---

### Welcome to LangChain: From Components to Composition

In the previous module, we built a RAG pipeline from scratch. We learned about the core components: document loaders, splitters, embedding models, vector stores, and LLMs. While building from scratch provides a deep understanding, it's not always the most efficient way to build production systems.

This is where **LangChain** comes in.

LangChain is a powerful framework for developing applications powered by language models. It provides a standard, extensible interface and a rich library of components that can be "chained" together to create sophisticated AI workflows.

In this notebook, we will take our first steps with LangChain. We will learn its fundamental building blocks and use the **LangChain Expression Language (LCEL)** to compose them into our first real pipeline. This "chain" will form the foundation of the Manufacturing Copilot, a tool designed to provide instant, accurate answers to technicians on the factory floor.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1.  **Understand LangChain's Core Abstractions:** Grasp the fundamental building blocks: `PromptTemplate`, `LLM`, and `OutputParser`.
2.  **Master LangChain Expression Language (LCEL):** Use the `|` (pipe) operator to compose components into a declarative, runnable pipeline.
3.  **Build a Simple RAG-style Chain:** Create a complete chain that takes a user's question and context to generate a helpful answer.
4.  **Implement a Governance Wrapper:** Build a Python function to log key metadata about your chain's execution, a critical step for creating auditable, production-ready AI systems.

## ⚙️ Step 1: Environment Setup

First, let's set up our environment. We'll need to install the core `langchain` library and the `langchain-openai` integration to use OpenAI's models.

We will also set up our OpenAI API key. For security, it's best practice to store this in an environment variable rather than hardcoding it in the notebook.

*(Note: If you are running this locally, you will need to have a `.env` file with your `OPENAI_API_KEY` or set it as an environment variable in your system.)*

In [ ]:
# Install necessary packages
!pip install -q langchain langchain-openai python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# It's recommended to set your OpenAI API key as an environment variable
# to avoid exposing it in your code.
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
    print("API Key not found. Please set it as an environment variable or replace 'YOUR_API_KEY'.")
else:
    print("OpenAI API Key loaded successfully from environment variables.")

## 🧱 Step 2: The Three Core Components of a LangChain Chain

At its core, a LangChain pipeline, or "chain," is composed of three fundamental building blocks. Understanding these is the key to mastering LangChain.

1.  **The `PromptTemplate`**: This is the blueprint for the instructions given to the LLM. It's a template that combines user input with fixed instructions to create a complete, precise prompt. This ensures that every query is sent to the LLM in a consistent and well-structured format.

2.  **The Model (`ChatOpenAI`)**: This is the "brain" of the operation. It can be a large language model (LLM) or a chat model. We'll use `ChatOpenAI` to connect to OpenAI's powerful models like GPT-4o-mini. LangChain provides dozens of integrations, so you can easily swap this component for a different model from Hugging Face, Anthropic, or Google.

3.  **The `OutputParser`**: This component takes the raw output from the LLM and transforms it into a more usable format. The simplest parser, `StrOutputParser`, takes the LLM's response and converts it into a clean string. More advanced parsers can convert the output into structured data like JSON or Python objects.

We will compose these three components using the **LangChain Expression Language (LCEL)**, which uses the pipe `|` symbol. This creates a declarative, elegant, and powerful way to define our AI pipeline.

`chain = prompt | model | output_parser`

Let's build our first chain.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pathlib import Path

# --- 1. Prepare Sample Data ---
# For this example, we'll create a dummy Standard Operating Procedure (SOP) document.
# This simulates the knowledge base our copilot will use.
SOP_DIR = Path('data/sop_manuals')
SOP_DIR.mkdir(parents=True, exist_ok=True)
SOP_PATH = SOP_DIR / 'punch_press_v7.txt'
SOP_PATH.write_text("""
Standard Operating Procedure: Punch Press v7

Safety First: Always wear safety glasses and gloves. Ensure the machine is fully locked out before performing any maintenance.

Lubrication Schedule:
- Daily: Check hydraulic fluid levels and top off if necessary.
- Weekly: Lubricate all punch press bearings with Grade 2 lithium grease.
- Monthly: Inspect and lubricate all other moving parts, including slides and guides.
""", encoding='utf-8')

# Load the content to be used as context
sample_context = SOP_PATH.read_text(encoding='utf-8')

# --- 2. Define the LangChain Components ---

# The Prompt Template: It structures the input for the LLM.
# We define placeholders for the dynamic parts of the prompt.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Manufacturing Copilot. Your role is to provide clear and concise answers based on Standard Operating Procedures."),
    ("human", "Please use the following context to answer the question.\n\nContext: {context}\n\nQuestion: {question}")
])

# The Model: We'll use OpenAI's gpt-4o-mini, a fast and capable model.
# `temperature=0` makes the output deterministic and factual.
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# The Output Parser: This ensures the final output is a simple string.
output_parser = StrOutputParser()


# --- 3. Compose the Chain with LCEL ---
# The pipe symbol `|` chains the components together.
# The data flows from left to right: prompt -> model -> parser.
chain = prompt_template | model | output_parser


# --- 4. Invoke the Chain ---
# We pass a dictionary with the required inputs to the `invoke` method.
question = "How often should I lubricate the punch press bearings?"
response = chain.invoke({
    "context": sample_context,
    "question": question
})

print(f"Question: {question}\n")
print(f"Response: {response}")

## 🔒 Step 3: Implementing a Governance Wrapper for Auditing

In a real-world manufacturing environment, simply getting an answer isn't enough. For compliance, safety, and quality control (e.g., under ISO 9001 standards), it's crucial to maintain a record of the AI's operations. Who asked what? What information was used? How long did it take?

To address this, we will build a "governance wrapper." This is a Python function that will:
1.  Execute our LangChain chain.
2.  Record critical metadata about the run.
3.  Log this metadata to a file, creating an immutable audit trail.

This practice is essential for building responsible and trustworthy AI systems. The log can be used to debug issues, analyze performance, and demonstrate compliance to auditors.

Our log will capture:
-   A unique hash of the prompt, input, and output to ensure data integrity.
-   The model used for the generation.
-   The latency of the response.
-   Contextual information like plant ID and supervisor.

In [ ]:
import json
from hashlib import sha256
from datetime import datetime
import time

def execute_and_log_chain(chain, inputs: dict):
    """
    Executes a LangChain chain, measures latency, and logs governance metadata.

    Args:
        chain: The LangChain chain to execute.
        inputs: The dictionary of inputs for the chain.

    Returns:
        A tuple containing the response and the evidence dictionary.
    """
    print("--- Executing Chain with Governance ---")
    start_time = time.time()
    
    # --- 1. Execute the chain ---
    response = chain.invoke(inputs)
    
    # --- 2. Calculate latency ---
    latency_ms = (time.time() - start_time) * 1000
    
    # --- 3. Create the governance record (evidence) ---
    # We hash the core components to create a verifiable signature for the run.
    run_evidence = {
        "timestamp": datetime.now().isoformat(),
        "plant_id": "PUNE-01",
        "shift": "morning",
        "supervisor_id": "SUP-4872",
        "prompt_hash": sha256(str(chain.first.pretty_repr()).encode()).hexdigest()[:12],
        "input_context_hash": sha256(inputs['context'].encode()).hexdigest()[:12],
        "output_hash": sha256(response.encode()).hexdigest()[:12],
        "model": chain.middle[0].model_name,
        "latency_ms": round(latency_ms, 2)
    }
    
    # --- 4. Log the evidence to a file ---
    # We use a JSON Lines format (.jsonl) for easy, append-only logging.
    log_dir = Path('governance_logs')
    log_dir.mkdir(exist_ok=True)
    log_file = log_dir / 'langchain_run_log.jsonl'
    
    with open(log_file, 'a') as f:
        f.write(json.dumps(run_evidence) + '\n')
        
    print(f"Successfully logged governance evidence to {log_file.name}")
        
    return response, run_evidence

# --- Execute the chain using our new governance wrapper ---
question = "How often should I lubricate the punch press bearings?"
inputs = {
    "context": sample_context,
    "question": question
}
final_response, evidence = execute_and_log_chain(chain, inputs)

print("\n--- Chain Response ---")
print(final_response)
print("\n--- Governance Log ---")
print(json.dumps(evidence, indent=2))

## ✅ Congratulations and Next Steps!

Congratulations! You have successfully built and executed your first LangChain pipeline.

You've learned about the three core components—**Prompts, Models, and Output Parsers**—and how to compose them into a powerful, declarative chain using the **LangChain Expression Language (LCEL)**. You also took a critical first step towards production-readiness by implementing a **governance wrapper** to create an audit trail of your chain's activity.

### Key Takeaways:
-   **Composition is Key:** LCEL's pipe `|` syntax is the heart of modern LangChain development. It allows you to build complex logic in a simple, readable way.
-   **Components are Swappable:** The modular nature of LangChain means you can easily swap out any component. You could change the `model` to one from Google or Hugging Face without altering the rest of your chain.
-   **Governance is Not an Afterthought:** Building logging and auditing into your process from the beginning is essential for creating responsible and reliable AI applications.

In the next notebook, **`02_message_structures.ipynb`**, we will dive deeper into the data structures that flow through these chains, exploring the different types of `Messages` that are used to manage conversations and instruct chat models. Let's continue building!